In [1]:
from imports import *

activation_embeds = {
        "layer 0": [],
        "layer 1": [],
        "layer 2": [],
        "layer 3": [],
        "layer 4": [],
        "layer 5": []
    }

/Users/maheepchaudhary/miniforge3/envs/transit/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
for k in activation_embeds.keys():
    print(k)

layer 0
layer 1
layer 2
layer 3
layer 4
layer 5


In [18]:
import numpy as np
from scipy.spatial.distance import euclidean

from fastdtw import fastdtw

x = np.array([[2,3,43,44,5]])
y = np.array([[2,3,43,44,5]])
distance, path = fastdtw(x, y, dist=euclidean)
print(distance)

0.0


In [2]:
_, path = fastdtw(reference, flattened_act, dist=euclidean)

NameError: name 'reference' is not defined

In [20]:
from imports import *
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
import numpy as np

def average_activation_for_layer(model_name):
    datasets = ["tinystories", "summarisation", "alpaca"]

    if model_name == "Pythia14m" or model_name == "Pythia70m":
        model_layer = 6
        activations = {f"layer {i}": [] for i in range(model_layer)}
    elif model_name == "Pythia160m":
        model_layer = 12
        activations = {f"layer {i}": [] for i in range(model_layer)}
    elif model_name == "Pythia410m":
        model_layer = 24
        activations = {f"layer {i}": [] for i in range(model_layer)}
    elif model_name == "Pythia1b":
        model_layer = 16
        activations = {f"layer {i}": [] for i in range(model_layer)}

    # Collect activations from datasets
    for dataset in datasets:
        file_path = f"data/{dataset}/{model_name}/activation_resid.pkl"
        if os.path.exists(file_path):
            with open(file_path, "rb") as f:
                data = pickle.load(f)
                print(f"Loaded {len(data)} activation entries for {dataset}")
                for count, act_data in enumerate(data):
                    if count < model_layer:
                        activations[f"layer {count}"].append(act_data)
        else:
            print(f"File not found for {dataset}: {file_path}")

    # Process activations
    avg_activations = {}
    for layer, acts in activations.items():
        if acts:
            reference = np.array(acts[1]).reshape(-1, 1)  # Flatten the reference activation
            reference_len = len(reference)
            aligned_acts = []

            for act in acts:
                flattened_act = np.array(act).reshape(-1, 1)  # Flatten each activation before DTW
                _, path = fastdtw(reference, flattened_act, dist=euclidean)

                # Align activations based on DTW path, creating an interpolated activation of reference length
                aligned_act = np.zeros((reference_len, 1))
                for ref_idx, act_idx in path:
                    aligned_act[ref_idx] += flattened_act[act_idx]
                
                # Normalize the aligned activations by the number of matches to account for repetitions
                counts = np.bincount([ref_idx for ref_idx, _ in path], minlength=reference_len)
                counts[counts == 0] = 1  # Avoid division by zero
                aligned_act /= counts.reshape(-1, 1)
                
                aligned_acts.append(aligned_act.flatten())

            # Compute the mean of the aligned activations
            avg_activations[layer] = np.mean(aligned_acts, axis=0)
        else:
            print(f"No activation data found for layer {layer}")

    return avg_activations

# Call the function
avg_activation = average_activation_for_layer("Pythia70m")

Loaded 6 activation entries for tinystories
Loaded 6 activation entries for summarisation
Loaded 6 activation entries for alpaca


In [21]:
for k, v in avg_activation.items():
    print(k, v.shape)

layer 0 (340,)
layer 1 (340,)
layer 2 (340,)
layer 3 (340,)
layer 4 (340,)
layer 5 (340,)


In [7]:
from imports import *

file_path = f"data/tinystories/Pythia70m/activation_resid.pkl"
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        data = pickle.load(f)
# for count, act_data in enumerate(data):
#     print(act_data)
print(data)
    

[[2.2553408 2.0688918 2.0613337 1.9471403 2.1523995 2.0567496 2.0541742
  2.0493457 2.0534885 2.0849519 2.0526545 2.1793873 2.1327612 2.0472004
  2.0115912 1.9700245 2.0006151 1.9769596 1.9683774 1.9847175 1.97068
  1.9524602 1.9494752 1.9669728 1.9559224 1.9521474 1.9524639 1.9496708
  1.9546218 1.9555161 1.9493585 1.948326  1.9472961 1.9591253 1.9475896
  1.951405  1.946959  1.9637146 1.9507781 1.9449279 1.9527779 1.9398063
  1.9430442 1.9545801 1.9469472 1.9434712 1.9387171 1.9412998 1.945561
  1.9422435 1.9492078 1.9465766 1.925903  1.92721   1.9274689 1.9477957
  1.9359568 1.9351714 1.9402848 1.9202613 1.941752  1.9520992 1.9392884
  1.9375138 1.9417018 1.9431307 1.9402736 1.93885   1.9333259 1.9269534
  1.9417624 1.9422526 1.9399033 1.9395934 1.944549  1.933958  1.9367216
  1.9455318 1.9408805 1.9493406 1.9311193 1.9411293 1.9391283 1.9442606
  1.9234191 1.9252918 1.9395732 1.9489162 1.9413918 1.9415876 1.9230908
  1.9294263 1.9374062 1.9549191 1.9488082 1.9400299 1.9420103 1.945